In [85]:
import pandas as pd
import json
import demjson3 as dem


In [100]:
df=pd.read_csv('Dataset/movies_dataset.csv', low_memory=False)
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [101]:
serie=pd.Series(data=df['belongs_to_collection'], index=df.index)

In [ ]:
#serie = serie.apply(lambda x: json.loads(x) if pd.notnull(x) else {})

In [102]:
# Iterar sobre cada elemento de la serie
for i, value in serie.items():
    if pd.notnull(value):  # Verificar si el valor no es nulo
        if isinstance(value, str):  # Verificar si el valor es una cadena
            try:
                # Analizar la cadena JSON utilizando dem
                collection_dict = dem.decode(value)
                if isinstance(collection_dict, dict):  # Verificar si el resultado es un objeto válido
                    # Obtener los valores deseados
                    collection_id = collection_dict.get('id')
                    collection_name = collection_dict.get('name')
                    # Asignar los valores a las nuevas columnas
                    df.loc[i, 'collection_id'] = collection_id
                    df.loc[i, 'collection_name'] = collection_name
                else:
                    # Manejar el caso en que no sea un objeto válido
                    df.loc[i, 'collection_id'] = None
                    df.loc[i, 'collection_name'] = None
            except dem.JSONDecodeError:
                # Manejar errores de decodificación JSON
                df.loc[i, 'collection_id'] = None
                df.loc[i, 'collection_name'] = None
        else:
            # Manejar otros tipos de valores
            df.loc[i, 'collection_id'] = None
            df.loc[i, 'collection_name'] = None
    else:
        # Manejar el caso en que el valor sea nulo
        df.loc[i, 'collection_id'] = None
        df.loc[i, 'collection_name'] = None

In [104]:
df=df.drop('belongs_to_collection', axis=1)
df=df.drop('homepage', axis=1)
df=df.drop('video', axis=1)
df=df.drop('spoken_languages', axis=1)

In [106]:
serie1=pd.Series(data=df['genres'], index=df.index)

In [108]:
serie1[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [112]:
serie1 = df['genres']

# Crea las nuevas columnas que deseas agregar
new_columns = ['genre_1', 'genre_2', 'genre_3']

# Itera sobre cada valor de la Serie 'serie1' y desanida los datos
for index, value in serie1.items():
    # Convierte el valor de la celda en una lista de diccionarios utilizando json.loads
    genre_list = json.loads(value)
    
    # Obtén los valores deseados de cada diccionario en la lista
    genre_values = [genre_dict['name'] for genre_dict in genre_list]
    
    # Rellena los valores en las nuevas columnas
    for i, column in enumerate(new_columns):
        if i < len(genre_values):
            df.at[index, column] = genre_values[i]

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

In [ ]:
# Obtén la longitud máxima de la lista de géneros en la Serie 'serie1'
max_genres = serie1.apply(lambda x: len(json.loads(x))).max()

# Crea las nuevas columnas utilizando el prefijo 'genre_' y el número de posición
new_columns = [f'genre_{i+1}' for i in range(max_genres)]

# Itera sobre cada valor de la Serie 'serie1' y desanida los datos
for index, value in serie1.items():
    # Convierte el valor de la celda en una lista de diccionarios utilizando json.loads
    genre_list = json.loads(value)
    
    # Obtén los nombres de los géneros en la lista
    genre_names = [genre_dict['name'] for genre_dict in genre_list]
    
    # Rellena los valores en las nuevas columnas
    for i, column in enumerate(new_columns):
        if i < len(genre_names):
            df.at[index, column] = genre_names[i]

In [114]:
def fix_json_data(data):
    try:
        return json.loads(data.replace("'", '"'))
    except:
        return []

serie1 = serie1.apply(fix_json_data)

In [115]:
# Desanidar la Serie 'serie1' y crear nuevas columnas
df_unnested = pd.json_normalize(serie1.explode(), sep='_')
df_unnested.columns = [f'genre_{col}' for col in df_unnested.columns]

# Concatenar el DataFrame original con el DataFrame desanidado
df_new = pd.concat([df, df_unnested], axis=1)

# Verificar el resultado


   adult    budget                                             genres     id  \
0  False  30000000  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...    862   
1  False  65000000  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   8844   
2  False         0  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...  15602   
3  False  16000000  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  31357   
4  False         0                     [{'id': 35, 'name': 'Comedy'}]  11862   

     imdb_id original_language               original_title  \
0  tt0114709                en                    Toy Story   
1  tt0113497                en                      Jumanji   
2  tt0113228                en             Grumpier Old Men   
3  tt0114885                en            Waiting to Exhale   
4  tt0113041                en  Father of the Bride Part II   

                                            overview popularity  \
0  Led by Woody, Andy's toys live happily in his ...  21.946943   
1  Whe

In [116]:

df_new

,adult,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,runtime,status,tagline,title,vote_average,vote_count,collection_id,collection_name,genre_id,genre_name
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,81.0,Released,NaN,Toy Story,7.7,5415.0,10194.0,Toy Story Collection,16.0,Animation
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,None,35.0,Comedy
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,119050.0,Grumpy Old Men Collection,10751.0,Family
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,None,12.0,Adventure
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,96871.0,Father of the Bride Collection,14.0,Fantasy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,Action
93544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,Drama
93545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,Thriller
93546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
